# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [252]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-e632f10b70-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Sport Workflow Series ("Running explorative search") 

Consider the following exploratory information need:

> investigate the association football players and find the main BGPs related to this sports. Compare awards and participation in the competitions of the players

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wd:Q5`       | Human         | node      |
| `wd:Q9124`    | Eliud Kipchoge| node      |
| `wd:Q853003`| athletics at the 2008 Summer Olympics – men's 200 metres |node|





Also consider

```
wd:Q9124 ?p ?obj .
```

is the BGP to retrieve all **properties of Eliud Kipchoge**


Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.


The workload should:


1. Identify the BGP for runner

2. Identify the BGP for long distance running

3. Return the disciplines of the long distance running (e.g. marathon) (the result set must be a list of couples discipline IRI and label).

4. Consider marathons ran in Europe. Return the number of different runners who won a marathon group by their country of citizenship (the result set must be a triples of country IRI, label and #different runners).

5. Identify the BGP for Olympic Games

6. Consider only the summer olympic games of 2004-2008-2012-2016. Return the number of marathon runners who had partecipated to all these editions group by their country of citizenship (if the runner partecipated in more than one edition of the game, consider him/her only once) (the result set must be a list of country IRI, label and #marathon runners)

7. For each disciplines of the long distance running find who hold the World Record (the result set must be a list of elements with discipline IRI, label and athlete IRI and label).

## Task 1
Identify the BGP for runner

In [250]:
# query example
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q9124 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1000'), ('name', 'record held')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P1146'), ('name', 'World Athletics athlete ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')]
[('p', 'http://www.wikidata.org/prop/direct/P1412'), ('name', 'languages spoken, written or signed')]
[('p', 'http://www.wikidata.org/prop/direct/P1447'), ('name', 'Sports-Reference.com Olympic athlete ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]
[('p', 'http://www.wikidata.org/prop/direct/P1559'), ('name', 'name in native language')]
[('p', 'http://www.wikidata.org/prop/direct/P172'), ('name', 'ethnic group')]
[('p', 'http://www.wikidata.org/prop/direct/P1741'), ('name', 'GTAA ID')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P19'), ('name', 

Final query for this task

In [7]:
# write your queries
# using occupation to find runner
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   # bind something
   wd:Q9124 wdt:P106 ?o .
   # get the label
   ?o sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q13382460'), ('name', 'marathon runner')]
[('o', 'http://www.wikidata.org/entity/Q11513337'), ('name', 'athletics competitor')]
[('o', 'http://www.wikidata.org/entity/Q4439155'), ('name', 'long-distance runner')]
3


In [14]:
# write your final query
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   # bind something
   wd:Q4439155 wdt:P279 ?o.
   # get the label
   ?o sc:name ?name.
   FILTER regex(?name , 'runner')
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)


Results
[('o', 'http://www.wikidata.org/entity/Q12803959'), ('name', 'runner')]
1


## Task 2
Identify the BGP for long distance running

In [19]:
# write your queries
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q9124 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1000'), ('name', 'record held')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P1146'), ('name', 'World Athletics athlete ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')]
[('p', 'http://www.wikidata.org/prop/direct/P1412'), ('name', 'languages spoken, written or signed')]
[('p', 'http://www.wikidata.org/prop/direct/P1447'), ('name', 'Sports-Reference.com Olympic athlete ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]
[('p', 'http://www.wikidata.org/prop/direct/P1559'), ('name', 'name in native language')]
[('p', 'http://www.wikidata.org/prop/direct/P172'), ('name', 'ethnic group')]
[('p', 'http://www.wikidata.org/prop/direct/P1741'), ('name', 'GTAA ID')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P19'), ('name', 

Final query for this task

In [18]:
# write your final query
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   # bind something
   wd:Q9124 wdt:P106 ?o .
   # get the label
   ?o sc:name ?name.
   FILTER regex(?name , 'long-distance runner')
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q4439155'), ('name', 'long-distance runner')]
1


## Task 3
Return the disciplines of the long distance running (e.g. marathon) (the result set must be a list of couples discipline IRI and label).

In [24]:
# write your queries
#find the related object
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   # bind something
   wd:Q4439155 ?p ?o.
   # get the label
   ?o sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q8928097'), ('name', 'Category:Long-distance runners')]
[('o', 'http://www.wikidata.org/entity/Q13382608'), ('name', 'cross-country skier')]
[('o', 'http://www.wikidata.org/entity/Q28640'), ('name', 'profession')]
[('o', 'http://www.wikidata.org/entity/Q917206'), ('name', 'long-distance running')]
[('o', 'http://www.wikidata.org/entity/Q12803959'), ('name', 'runner')]
5


Final query for this task

In [44]:
# write your final query
queryString = """
SELECT DISTINCT ?o  ?name 
WHERE {
   # bind something
   wd:Q917206 wdt:P527 ?o.
   # get the label
   ?o sc:name ?name.
   
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q215677'), ('name', 'half marathon')]
[('o', 'http://www.wikidata.org/entity/Q163892'), ('name', '10,000 metres')]
[('o', 'http://www.wikidata.org/entity/Q240500'), ('name', '5000 metres')]
[('o', 'http://www.wikidata.org/entity/Q40244'), ('name', 'marathon')]
[('o', 'http://www.wikidata.org/entity/Q1202167'), ('name', 'ekiden')]
[('o', 'http://www.wikidata.org/entity/Q2093999'), ('name', 'trail running')]
[('o', 'http://www.wikidata.org/entity/Q2774730'), ('name', '10K run')]
[('o', 'http://www.wikidata.org/entity/Q500050'), ('name', 'cross country running')]
[('o', 'http://www.wikidata.org/entity/Q9171043'), ('name', '5K run')]
9


## Task 4
Consider marathons ran in Europe. Return the number of different runners who won a marathon group by their country of citizenship (the result set must be a triples of country IRI, label and #different runners).

Final query for this task

In [76]:
# write your queries
#find the related subject 
queryString = """
SELECT DISTINCT ?s ?name 
WHERE {
   # bind something
   ?s ?p wd:Q40244.
   # get the label
   ?s sc:name ?name.
   FILTER regex(?name , 'Europe')
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q55780710'), ('name', "2018 European Championships in Athletics – men's marathon")]
[('s', 'http://www.wikidata.org/entity/Q17496398'), ('name', "2014 European Athletics Championships – Women's Marathon")]
[('s', 'http://www.wikidata.org/entity/Q4494720'), ('name', "1966 European Athletics Championships – men's marathon")]
[('s', 'http://www.wikidata.org/entity/Q4606142'), ('name', "2006 European Athletics Championships – Women's Marathon")]
[('s', 'http://www.wikidata.org/entity/Q3229843'), ('name', "1958 European Athletics Championships – men's marathon")]
[('s', 'http://www.wikidata.org/entity/Q3429452'), ('name', "1982 European Athletics Championships – Women's Marathon")]
[('s', 'http://www.wikidata.org/entity/Q2537624'), ('name', '1998 European Athletics Championships – Marathon')]
[('s', 'http://www.wikidata.org/entity/Q3652508'), ('name', "1998 European Athletics Championships – Men's Marathon")]
[('s', 'http://www.wikidata.org/ent

In [77]:
# write your queries 

queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q55780710 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('name', 'follows')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P2094'), ('name', 'competition class')]
[('p', 'http://www.wikidata.org/prop/direct/P2671'), ('name', 'Google Knowledge Graph ID')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('name', 'location')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('name', 'part of')]
[('p', 'http://www.wikidata.org/prop/direct/P585'), ('name', 'point in time')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport')]
10


In [84]:
# write your queries 
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   # bind something
   ?s ?p wd:Q40244.
   # get the label
   ?s sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q1701191'), ('name', 'John Nada Saya')]
[('s', 'http://www.wikidata.org/entity/Q16006623'), ('name', 'Harry Payne')]
[('s', 'http://www.wikidata.org/entity/Q16007889'), ('name', 'John Tarrant')]
[('s', 'http://www.wikidata.org/entity/Q16008400'), ('name', 'Waldemar Karlsson')]
[('s', 'http://www.wikidata.org/entity/Q1602744'), ('name', 'Hellen Mugo')]
[('s', 'http://www.wikidata.org/entity/Q16185418'), ('name', 'Jerry Kiernan')]
[('s', 'http://www.wikidata.org/entity/Q16210841'), ('name', 'Elva Dryer')]
[('s', 'http://www.wikidata.org/entity/Q16212704'), ('name', 'José Alejandro Semprún')]
[('s', 'http://www.wikidata.org/entity/Q16226788'), ('name', 'Jacob Bragg')]
[('s', 'http://www.wikidata.org/entity/Q17010986'), ('name', 'Naomi Sakashita')]
[('s', 'http://www.wikidata.org/entity/Q17021963'), ('name', 'Niagara Falls International Marathon')]
[('s', 'http://www.wikidata.org/entity/Q17051087'), ('name', 'Kaohsiung International Marathon')

In [253]:
# write your queries
queryString = """
SELECT DISTINCT ?WIN_CIT ?WIN_CIT_NAME ?loc   (count(?winner) as ?win)
WHERE {
    # bind something
   
    ?winner wdt:P27 ?WIN_CIT.
    ?WIN_CIT sc:name ?WIN_CIT_NAME .
    {
        SELECT DISTINCT ?winner ?winner_name
            WHERE {
                #bind something
                ?winner wdt:P276 ?loc.
                ?europeanmarathon ?p wd:Q40244 .
                ?europeanmarathon wdt:P17 ?countrymarathon .
                ?countrymarathon wdt:P30 wd:Q46 .
                #get label
                ?europeanmarathon sc:name ?name .
                ?europeanmarathon wdt:P1346 ?winner .
                ?winner sc:name ?winner_name .
               
            }

    }
    
}
 GROUP BY ?WIN_CIT ?WIN_CIT_NAME
limit 20
"""
print("Results")
x=run_query(queryString)

Results
The operation failed QueryBadFormed: a bad request has been sent to the endpoint, probably the sparql query is bad formed. 

Response:
b'Virtuoso 37000 Error SP031: SPARQL compiler: Variable ?loc is used in the result set outside aggregate and not mentioned in GROUP BY clause\n\nSPARQL query:\ndefine sql:big-data-const 0\n#output-format:application/sparql-results+json\n\n##-e632f10b70-##\nPREFIX wd: <http://www.wikidata.org/entity/> \nPREFIX wdt: <http://www.wikidata.org/prop/direct/> \nPREFIX sc: <http://schema.org/>\n\n\nSELECT DISTINCT ?WIN_CIT ?WIN_CIT_NAME ?loc   (count(?winner) as ?win)\nWHERE {\n    # bind something\n   \n    ?winner wdt:P27 ?WIN_CIT.\n    ?WIN_CIT sc:name ?WIN_CIT_NAME .\n    {\n        SELECT DISTINCT ?winner ?winner_name\n            WHERE {\n                #bind something\n                ?winner wdt:P276 ?loc.\n                ?europeanmarathon ?p wd:Q40244 .\n                ?europeanmarathon wdt:P17 ?countrymarathon .\n                ?countrymar

## Task 5
Identify the BGP for Olympic Games

In [42]:
# write your queries 
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q9124 ?p ?o.
   # get the label
   ?p sc:name ?name.
      
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1000'), ('name', 'record held')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P1146'), ('name', 'World Athletics athlete ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')]
[('p', 'http://www.wikidata.org/prop/direct/P1412'), ('name', 'languages spoken, written or signed')]
[('p', 'http://www.wikidata.org/prop/direct/P1447'), ('name', 'Sports-Reference.com Olympic athlete ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]
[('p', 'http://www.wikidata.org/prop/direct/P1559'), ('name', 'name in native language')]
[('p', 'http://www.wikidata.org/prop/direct/P172'), ('name', 'ethnic group')]
[('p', 'http://www.wikidata.org/prop/direct/P1741'), ('name', 'GTAA ID')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P19'), ('name', 

In [43]:
# write your queries 
#find the related object
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   # bind something
   wd:Q9124 wdt:P641 ?o.
   # get the label
   ?o sc:name ?name.
     
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q542'), ('name', 'athletics')]
1


Final query for this task

In [47]:
# write your queries
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q542 ?p ?o.
   # get the label
   ?p sc:name ?name.
   
}
LIMIT 40
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2869'), ('name', 'record or record progression')]
[('p', 'http://www.wikidata.org/prop/direct/P1014'), ('name', 'Art & Architecture Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1036'), ('name', 'Dewey Decimal Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('name', "topic's main Wikimedia portal")]
[('p', 'http://www.wikidata.org/prop/direct/P1225'), ('name', 'U.S. National Archives Identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1245'), ('name', 'OmegaWiki Defined Meaning')]
[('p', 'http://www.wikidata.org/prop/direct/P1282'), ('name', 'OpenStreetMap tag or key')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct

In [50]:
# write your queries
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   # bind something
   wd:Q542 wdt:P279|wdt:P527 ?o.
   # get the label
   ?o sc:name ?name.
   
}
LIMIT 40
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q3312129'), ('name', 'track and field')]
[('o', 'http://www.wikidata.org/entity/Q3216963'), ('name', 'throwing event')]
[('o', 'http://www.wikidata.org/entity/Q11047409'), ('name', 'athletic culture')]
[('o', 'http://www.wikidata.org/entity/Q21467672'), ('name', 'running disciplines of track and field')]
[('o', 'http://www.wikidata.org/entity/Q10235779'), ('name', 'indoor athletics')]
[('o', 'http://www.wikidata.org/entity/Q202391'), ('name', 'racewalking')]
[('o', 'http://www.wikidata.org/entity/Q212434'), ('name', 'Olympic sport')]
[('o', 'http://www.wikidata.org/entity/Q3924425'), ('name', 'combined track and field events')]
[('o', 'http://www.wikidata.org/entity/Q52177345'), ('name', 'technical disciplines of athletics')]
[('o', 'http://www.wikidata.org/entity/Q1151752'), ('name', 'jumping')]
10


In [53]:
# write your final query
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   # bind something
   wd:Q212434 ?p ?o.
   # get the label
   ?o sc:name ?name.
   Filter regex(?name , 'Olympic Games')
   
}
LIMIT 40
"""

print("Results")
x=run_query(queryString)


Results
[('o', 'http://www.wikidata.org/entity/Q5389'), ('name', 'Olympic Games')]
1


## Task 6
Consider only the summer olympic games of 2004-2008-2012-2016. Return the number of marathon runners who had partecipated to all these editions group by their country of citizenship (if the runner partecipated in more than one edition of the game, consider him/her only once) (the result set must be a list of country IRI, label and #marathon runners)

In [88]:
# write your queries
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   # bind something
   wd:Q5389 ?p ?o.
   # get the label
   ?o sc:name ?name.
   
   
}
LIMIT 40
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q1093409'), ('name', 'Citius, Altius, Fortius')]
[('o', 'http://www.wikidata.org/entity/Q14624058'), ('name', 'Olympic flag')]
[('o', 'http://www.wikidata.org/entity/Q159821'), ('name', 'Summer Olympic Games')]
[('o', 'http://www.wikidata.org/entity/Q167170'), ('name', 'multi-sport event')]
[('o', 'http://www.wikidata.org/entity/Q18608583'), ('name', 'recurring sporting event')]
[('o', 'http://www.wikidata.org/entity/Q19180675'), ('name', 'Small Brockhaus and Efron Encyclopedic Dictionary')]
[('o', 'http://www.wikidata.org/entity/Q19190511'), ('name', 'New Encyclopedic Dictionary')]
[('o', 'http://www.wikidata.org/entity/Q1983209'), ('name', 'Category:Olympic Games')]
[('o', 'http://www.wikidata.org/entity/Q212434'), ('name', 'Olympic sport')]
[('o', 'http://www.wikidata.org/entity/Q221956'), ('name', 'olympiad')]
[('o', 'http://www.wikidata.org/entity/Q30059240'), ('name', "Real'nyj slovar' klassicheskih drevnostej po Ljubkeru")]
[('o', '

In [63]:
# write your final query
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   # bind something
   ?s ?p wd:Q5389.
   # get the label
   ?s sc:name ?name.
      
}
LIMIT 40
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q8695307'), ('name', 'Category:Olympic badminton players of Poland')]
[('s', 'http://www.wikidata.org/entity/Q16212726'), ('name', 'Indian women at the Olympics')]
[('s', 'http://www.wikidata.org/entity/Q8204577'), ('name', 'Category:2016 Summer Olympics')]
[('s', 'http://www.wikidata.org/entity/Q2748206'), ('name', 'Beijing 2008')]
[('s', 'http://www.wikidata.org/entity/Q3890208'), ('name', 'Olympic medal table')]
[('s', 'http://www.wikidata.org/entity/Q938381'), ('name', 'bids for Olympic Games')]
[('s', 'http://www.wikidata.org/entity/Q1093409'), ('name', 'Citius, Altius, Fortius')]
[('s', 'http://www.wikidata.org/entity/Q159821'), ('name', 'Summer Olympic Games')]
[('s', 'http://www.wikidata.org/entity/Q1983209'), ('name', 'Category:Olympic Games')]
[('s', 'http://www.wikidata.org/entity/Q212434'), ('name', 'Olympic sport')]
[('s', 'http://www.wikidata.org/entity/Q221956'), ('name', 'olympiad')]
[('s', 'http://www.wikidata.org/entity/Q

Final query for this task

In [85]:
# write your final query
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   # bind something
   wd:Q159821 ?p ?o.
   # get the label
   ?o sc:name ?name.
      
}
LIMIT 40
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q8098'), ('name', '1904 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q8150'), ('name', '1936 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q8415'), ('name', '1960 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q181278'), ('name', '2020 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q18608583'), ('name', 'recurring sporting event')]
[('o', 'http://www.wikidata.org/entity/Q40970'), ('name', 'International Olympic Committee')]
[('o', 'http://www.wikidata.org/entity/Q5389'), ('name', 'Olympic Games')]
[('o', 'http://www.wikidata.org/entity/Q8088'), ('name', '1900 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q82414'), ('name', 'Winter Olympic Games')]
[('o', 'http://www.wikidata.org/entity/Q8438'), ('name', '1972 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q8444'), ('name', '1976 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q8470'), ('name', '1988 Summer Olym

In [91]:
# write your final query
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
    ?s ?p wd:Q8558.
   # get the label
   ?p sc:name ?name.
      
}
LIMIT 40
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P6291'), ('name', 'advertises')]
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('name', 'field of work')]
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('name', 'facet of')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('name', 'follows')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('name', 'followed by')]
[('p', 'http://www.wikidata.org/prop/direct/P301'), ('name', "category's main topic")]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('name', 'part of')]
[('p', 'http://www.wikidata.org/prop/direct/P466'), ('name', 'occupant')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part')]
[('p', 'http://www.wikidata.org/prop/direct/P793'), ('name', 'significant event')]
[('p', 'http://www.wikidata.org/prop/direct/P971'), ('name', 'category combines topics')]
12


In [105]:
# write your final query
queryString = """
SELECT DISTINCT ?s ?name ?time
WHERE {
   # bind something
   ?s  wdt:P31 wd:Q159821;
       wdt:P585 ?time.
   # get the label
   ?s sc:name ?name.
    FILTER(YEAR(?time) >= 2004 && YEAR(?time) <=2016)
   
}
LIMIT 40
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q8558'), ('name', '2004 Summer Olympics'), ('time', '2004-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q8567'), ('name', '2008 Summer Olympics'), ('time', '2008-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q8577'), ('name', '2012 Summer Olympics'), ('time', '2012-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q8613'), ('name', '2016 Summer Olympics'), ('time', '2016-01-01T00:00:00Z')]
4


In [150]:
# write your final query
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   # bind something
   wd:Q8558 wdt:P527 ?o.
   # get the label
   ?o sc:name ?name.
   FILTER regex(?name , 'athletics')
 
}
LIMIT 40
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q339283'), ('name', 'athletics at the 2004 Summer Olympics')]
1


In [160]:
# write your queries
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   # bind something
    ?s wdt:P527 ?o.
   # get the label
   ?o sc:name ?name.
   FILTER (?s IN(wd:Q8558,wd:Q8567,wd:Q8577,wd:Q8613))
   FILTER regex(?name , 'athletics')
   
}
LIMIT 40
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q18193712'), ('name', 'athletics at the 2016 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q185262'), ('name', 'athletics at the 2012 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q339283'), ('name', 'athletics at the 2004 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q189941'), ('name', 'athletics at the 2008 Summer Olympics')]
4


In [163]:
# write your queries
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   # bind something
   ?s wdt:P527 ?o.
   # get the label
   ?o sc:name ?name.
   FILTER (?s IN(wd:Q18193712,wd:Q185262,wd:Q339283,wd:Q189941))
   FILTER regex(?name , 'marathon')
  
   
}
LIMIT 40
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q21858018'), ('name', "athletics at the 2016 Summer Olympics – men's marathon")]
[('o', 'http://www.wikidata.org/entity/Q2000988'), ('name', "athletics at the 2012 Summer Olympics – women's marathon")]
[('o', 'http://www.wikidata.org/entity/Q21858030'), ('name', "athletics at the 2016 Summer Olympics – women's marathon")]
[('o', 'http://www.wikidata.org/entity/Q2552272'), ('name', "athletics at the 2004 Summer Olympics – women's marathon")]
[('o', 'http://www.wikidata.org/entity/Q693595'), ('name', "athletics at the 2008 Summer Olympics – men's marathon")]
[('o', 'http://www.wikidata.org/entity/Q947691'), ('name', "athletics at the 2008 Summer Olympics – women's marathon")]
[('o', 'http://www.wikidata.org/entity/Q1798983'), ('name', "athletics at the 2012 Summer Olympics – men's marathon")]
[('o', 'http://www.wikidata.org/entity/Q2443986'), ('name', "athletics at the 2004 Summer Olympics – men's marathon")]
8


In [254]:
# write your queries
queryString = """
SELECT DISTINCT ?s ?name 
WHERE {
   # bind something
   ?s wdt:P1344 ?o.
   # get the label
   ?s sc:name ?name.
   FILTER (?o IN(wd:Q21858018,wd:Q2000988,wd:Q21858030,wd:Q2552272,wd:Q693595,wd:Q947691,wd:Q1798983,wd:Q2443986))
}
LIMIT 40
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q26221036'), ('name', 'Marhu Teferi')]
[('s', 'http://www.wikidata.org/entity/Q4376764'), ('name', 'Abdi Nageeye')]
[('s', 'http://www.wikidata.org/entity/Q9124'), ('name', 'Eliud Kipchoge')]
[('s', 'http://www.wikidata.org/entity/Q6373165'), ('name', 'Karolina Jarzyńska')]
[('s', 'http://www.wikidata.org/entity/Q7367027'), ('name', 'Rosa Chacha')]
[('s', 'http://www.wikidata.org/entity/Q26472794'), ('name', 'Luis Orta')]
[('s', 'http://www.wikidata.org/entity/Q526508'), ('name', 'Helalia Johannes')]
[('s', 'http://www.wikidata.org/entity/Q19365752'), ('name', 'Tadesse Abraham')]
[('s', 'http://www.wikidata.org/entity/Q16212038'), ('name', 'Paul Pollock')]
[('s', 'http://www.wikidata.org/entity/Q24034036'), ('name', 'Volha Mazuronak')]
[('s', 'http://www.wikidata.org/entity/Q1702'), ('name', 'Stephen Kiprotich')]
[('s', 'http://www.wikidata.org/entity/Q21028954'), ('name', 'Andrea Deelstra')]
[('s', 'http://www.wikidata.org/entity/Q63691')

In [207]:
# write your queries
queryString = """
SELECT DISTINCT ?s  ?country_name
WHERE {
   # bind something
   ?s wdt:P1344 ?o.
   ?athlete wdt:P106 ?occupation .
   ?athlete wdt:P2416 ?sportcompeted .
   ?athlete wdt:P641 ?sport .
   ?s wdt:P27 ?country.
   # get the label         
   ?s sc:name ?name.
   ?country sc:name ?country_name.
   FILTER (?occupation=wd:Q13382460 || ?sport=wd:Q40244 || ?sportcompeted=wd:Q40244 ) .
   FILTER (?o IN(wd:Q21858018,wd:Q2000988,wd:Q21858030,wd:Q2552272,wd:Q693595,wd:Q947691,wd:Q1798983,wd:Q2443986))
   
}
LIMIT 40
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q26221036'), ('country_name', 'Israel')]
[('s', 'http://www.wikidata.org/entity/Q4376764'), ('country_name', 'Kingdom of the Netherlands')]
[('s', 'http://www.wikidata.org/entity/Q9124'), ('country_name', 'Kenya')]
[('s', 'http://www.wikidata.org/entity/Q26472794'), ('country_name', 'Venezuela')]
[('s', 'http://www.wikidata.org/entity/Q19365752'), ('country_name', 'Switzerland')]
[('s', 'http://www.wikidata.org/entity/Q16212038'), ('country_name', 'Ireland')]
[('s', 'http://www.wikidata.org/entity/Q1702'), ('country_name', 'Uganda')]
[('s', 'http://www.wikidata.org/entity/Q63691'), ('country_name', 'United States of America')]
[('s', 'http://www.wikidata.org/entity/Q22276905'), ('country_name', 'Paraguay')]
[('s', 'http://www.wikidata.org/entity/Q2340625'), ('country_name', 'Belgium')]
[('s', 'http://www.wikidata.org/entity/Q26483251'), ('country_name', 'Mongolia')]
[('s', 'http://www.wikidata.org/entity/Q26472786'), ('country_name', 'Peru

In [210]:
# write your queries
queryString = """
SELECT DISTINCT ?s  ?country_name (COUNT(?s) as ?numberathletes)
WHERE {
   # bind something
   ?s wdt:P1344 ?o.
   ?s wdt:P106 ?occupation .
   ?s wdt:P2416 ?sportcompeted .
   ?s wdt:P641 ?sport .
   ?s wdt:P27 ?country.
   # get the label
   ?s sc:name ?name.
   ?country sc:name ?country_name.
   FILTER (?occupation=wd:Q13382460 || ?sport=wd:Q40244 || ?sportcompeted=wd:Q40244 ) .
   FILTER (?o IN(wd:Q21858018,wd:Q2000988,wd:Q21858030,wd:Q2552272,wd:Q693595,wd:Q947691,wd:Q1798983,wd:Q2443986))
}
LIMIT 40
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q26464315'), ('country_name', 'South Korea'), ('numberathletes', '4')]
[('s', 'http://www.wikidata.org/entity/Q7367027'), ('country_name', 'Ecuador'), ('numberathletes', '4')]
[('s', 'http://www.wikidata.org/entity/Q1702'), ('country_name', 'Uganda'), ('numberathletes', '6')]
[('s', 'http://www.wikidata.org/entity/Q7610012'), ('country_name', 'South Africa'), ('numberathletes', '2')]
[('s', 'http://www.wikidata.org/entity/Q22002466'), ('country_name', 'United States of America'), ('numberathletes', '2')]
[('s', 'http://www.wikidata.org/entity/Q769607'), ('country_name', 'East Timor'), ('numberathletes', '2')]
[('s', 'http://www.wikidata.org/entity/Q526508'), ('country_name', 'Namibia'), ('numberathletes', '3')]
[('s', 'http://www.wikidata.org/entity/Q13554449'), ('country_name', 'Brazil'), ('numberathletes', '4')]
[('s', 'http://www.wikidata.org/entity/Q2271929'), ('country_name', 'Mongolia'), ('numberathletes', '8')]
[('s', 'http://www.wi

In [222]:
# write your queries
queryString = """
SELECT DISTINCT ?s ?country ?country_name (COUNT(?s) as ?numberathletes)
WHERE {
   # bind something
   ?s wdt:P1344 ?o.
   ?s wdt:P106 ?occupation .
   ?s wdt:P2416 ?sportcompeted .
   ?s wdt:P641 ?sport .
   ?s wdt:P27 ?country.
   # get the label
   ?s sc:name ?name.
   ?country sc:name ?country_name.
   FILTER (?occupation=wd:Q13382460 || ?sport=wd:Q40244 || ?sportcompeted=wd:Q40244 ) .
   FILTER (?o IN(wd:Q21858018,wd:Q2000988,wd:Q21858030,wd:Q2552272,wd:Q693595,wd:Q947691,wd:Q1798983,wd:Q2443986))
 
} 

LIMIT 40
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q20528590'), ('country', 'http://www.wikidata.org/entity/Q191'), ('country_name', 'Estonia'), ('numberathletes', '1')]
[('s', 'http://www.wikidata.org/entity/Q19365752'), ('country', 'http://www.wikidata.org/entity/Q39'), ('country_name', 'Switzerland'), ('numberathletes', '1')]
[('s', 'http://www.wikidata.org/entity/Q13554449'), ('country', 'http://www.wikidata.org/entity/Q155'), ('country_name', 'Brazil'), ('numberathletes', '4')]
[('s', 'http://www.wikidata.org/entity/Q6373165'), ('country', 'http://www.wikidata.org/entity/Q36'), ('country_name', 'Poland'), ('numberathletes', '2')]
[('s', 'http://www.wikidata.org/entity/Q6539472'), ('country', 'http://www.wikidata.org/entity/Q408'), ('country_name', 'Australia'), ('numberathletes', '1')]
[('s', 'http://www.wikidata.org/entity/Q441189'), ('country', 'http://www.wikidata.org/entity/Q419'), ('country_name', 'Peru'), ('numberathletes', '4')]
[('s', 'http://www.wikidata.org/entity/Q1702'), (

## Task 7
For each disciplines of the long distance running find who hold the World Record (the result set must be a list of elements with discipline IRI, label and athlete IRI and label).

In [223]:
# write your queries
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
    # bind something
    wd:Q4439155 ?p ?o .
    # get the label
    ?p sc:name ?name .
}
LIMIT 100
"""
print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P2671'), ('name', 'Google Knowledge Graph ID')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('name', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P425'), ('name', 'field of this occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('name', "topic's main category")]
7


In [224]:
# write your queries
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
    # bind something
    wd:Q4439155 wdt:P425 ?o .
    # get the label
    ?o sc:name ?name .
}
LIMIT 100
"""
print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q917206'), ('name', 'long-distance running')]
1


In [227]:
# write your queries
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
    # bind something
    ?s wdt:P425 wd:Q917206.
    # get the label
    ?s sc:name ?name .
}
LIMIT 100
"""
print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q4439155'), ('name', 'long-distance runner')]
1


Final query for this task

In [251]:
# write your final query
#from task 2 P1000 - 
queryString = """
SELECT DISTINCT ?discipline ?discipline_label ?athlete ?athlete_name 
WHERE {
    # bind something
    ?discipline wdt:P279 wd:Q4439155.
    ?athlete wdt:P1000 ?record;
            wdt:P106 ?discipline.
    # get the label
    ?athlete sc:name ?athlete_name.
    ?record sc:name ?discipline_label.
    FILTER REGEX(?discipline_label, 'world')
}
limit 20
"""
print("Results")
x=run_query(queryString)

Results
[('discipline', 'http://www.wikidata.org/entity/Q13382460'), ('discipline_label', 'Marathon world record progression'), ('athlete', 'http://www.wikidata.org/entity/Q1044585'), ('athlete_name', 'Caroline Walker')]
[('discipline', 'http://www.wikidata.org/entity/Q13382460'), ('discipline_label', 'Marathon world record progression'), ('athlete', 'http://www.wikidata.org/entity/Q171500'), ('athlete_name', 'Gebrselassie Haile')]
[('discipline', 'http://www.wikidata.org/entity/Q13382460'), ('discipline_label', 'Marathon world record progression'), ('athlete', 'http://www.wikidata.org/entity/Q728634'), ('athlete_name', 'Robert de Castella')]
[('discipline', 'http://www.wikidata.org/entity/Q13382460'), ('discipline_label', 'Marathon world record progression'), ('athlete', 'http://www.wikidata.org/entity/Q9124'), ('athlete_name', 'Eliud Kipchoge')]
[('discipline', 'http://www.wikidata.org/entity/Q13382460'), ('discipline_label', 'Marathon world record progression'), ('athlete', 'http://